2nd session

In [0]:
dbutils.help()

In [0]:
dbutils.credentials.help()

In [0]:
dbutils.fs.help()

In [0]:
dbutils.fs.ls('/')

In [0]:
dbutils.fs.ls('/databricks-datasets/')

In [0]:
dbutils.fs.ls('/databricks-datasets/online_retail/')

In [0]:
dbutils.fs.ls('/databricks-datasets/online_retail/data-001/data.csv')

In [0]:
df = spark.read.csv('/databricks-datasets/online_retail/data-001/data.csv', header=True,inferSchema=True)

In [0]:
df1 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/databricks-datasets/online_retail/data-001/data.csv")

In [0]:
display(df1)

In [0]:
dbutils.data.help()

In [0]:
dbutils.data.summarize(df1)

In [0]:
dbutils.fs.ls('/FileStore/')

In [0]:
dbutils.fs.ls('/FileStore/tables')

In [0]:
netflix_data = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/netflix_titles.csv")

In [0]:
display(netflix_data)

In [0]:
%sql
select count(distinct title),year(release_year) from netflix_titles_csv group by release_year

Databricks data profile. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import *
from pyspark.sql import functions
dataframe = spark.createDataFrame(
    data=[("1", "2025-05-02")],
    schema=["id", "date"]
)

In [0]:
dataframe

In [0]:
netflix_data.printSchema()

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

netflix_schema = StructType(fields=[StructField("show_id",StringType(),False),
                                    StructField("type",StringType(),True),
                                    StructField("title",StringType(),True),
                                    StructField("director",StringType(),True),
                                    StructField("cast",StringType(),True),
                                    StructField("country",StringType(),True),
                                    StructField("date_added",StringType(),True),
                                    StructField("release_year",StringType(),True),
                                    StructField("rating",StringType(),True),
                                    StructField("duration",StringType(),True),
                                    StructField("listed_in",StringType(),True),
                                    StructField("description",StringType(),True)])

In [0]:
output_path = "/FileStore/tables/databricks/netflix_titles.parquet"

In [0]:
netflix_data.write.mode("overwrite").parquet(output_path)

In [0]:
netflix_data_parquet = spark.read.parquet(output_path)

In [0]:

display(netflix_data_parquet)

In [0]:
pit_stops_data = spark.read.format("json").option("multiline", "true").load("/FileStore/tables/pit_stops.json")

In [0]:
display(pit_stops_data)

In [0]:
pit_stops_data.printSchema()

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

pit_stops_schema = StructType(fields=[StructField("driverId", IntegerType(), True),
                                       StructField("raceId", IntegerType(), False),
                                       StructField("stop", IntegerType(), True),
                                       StructField("time", StringType(), True),
                                       StructField("duration", StringType(), True),
                                       StructField("milliseconds", IntegerType(), True),
                                       StructField("lap", IntegerType(), True)])

In [0]:
pit_stops_df = spark.read.schema(pit_stops_schema).option("multiline", "true").json("/FileStore/tables/pit_stops.json")

In [0]:
display(pit_stops_df)

In [0]:
pit_stops_df.show()

3rd session

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("RDD Examples").getOrCreate()

In [0]:
data = spark.sparkContext.parallelize([1,2,3,4,5])

In [0]:
type(data)

In [0]:
data.collect()

In [0]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/annual_enterprise_survey_2023_financial_year_provisional.csv")

In [0]:
rdd = df.rdd

In [0]:
for row in rdd.collect():
    print(row)

In [0]:
industry_counts = rdd.map(lambda row: row['Industry_name_NZSIOC']).countByValue()
print(industry_counts)

In [0]:
value_rdd = rdd.map(lambda row: float(row['Value']) if row['Value'].replace('.', '', 1).isdigit() else None)
value_rdd = value_rdd.filter(lambda x: x is not None)
mean_total = value_rdd.mean()
print(mean_total)


In [0]:
rdd = sc.textFile("/FileStore/tables/annual_enterprise_survey_2023_financial_year_provisional.csv").filter(lambda row:"Financal ratios" not in row)
rdd = rdd.map(lambda row: row.split(","))

In [0]:
rdd.count()

In [0]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/annual_enterprise_survey_2023_financial_year_provisional.csv")

In [0]:
df.show(2)

In [0]:
df.filter(df["year"] == "2023").count()

In [0]:
df.select("Year","Industry_code_ANZSIC06").show()

In [0]:
df.groupBy("Year","Variable_category").agg({"Value":"sum"}).show()

In [0]:
df.printSchema()

In [0]:
%scala

case class FinancialData(
  Year: String,
  Industry_aggregation_NZSIOC: String,
  Industry_code_NZSIOC: String,
  Industry_name_NZSIOC: String,
  Units: String,
  Variable_code: String,
  Variable_name: String,
  Variable_category: String,
  Value: String,
  Industry_code_ANZSIC06: String
)

val dataset = spark.read.option("header", true).csv("/FileStore/tables/annual_enterprise_survey_2023_financial_year_provisional.csv").as[FinancialData]

In [0]:
%scala

dataset.groupBy("Year").count().show()

In [0]:
import pyspark.sql.functions as F

(spark.read
 .option("header", "true")
 .csv("/FileStore/tables/annual_enterprise_survey_2023_financial_year_provisional.csv")
 .filter(F.col("Variable_name") == "Total income")
        .withColumn("Value", F.col("Value").cast("float"))
        .select("Value","Variable_category")
        .groupBy("Variable_category").agg(F.mean("Value"))
                                          .explain())

In [0]:
import pyspark.sql.functions as F

(spark.read
 .option("header", "true")
 .csv("/FileStore/tables/annual_enterprise_survey_2023_financial_year_provisional.csv")
 .filter(F.col("Variable_name") == "Total income")
 .withColumn("Value", F.col("Value").cast("float"))
 .select("Value","Variable_category")
 .groupBy("Variable_category")
 .agg(F.mean("Value").alias("Mean_Value"))
 .filter(F.col("Variable_category") == "Financial positions")
 .explain())

5th sessions

In [0]:
%sql
CREATE SCHEMA hive_metastore.bronze

In [0]:
%sql
CREATE TABLE hive_metastore.bronze.employee(
  emp_id INT,
  emp_name STRING,
  emp_age INT,
  emp_dept STRING
);

In [0]:
%sql
describe extended hive_metastore.bronze.employee;

In [0]:
%sql

INSERT INTO hive_metastore.bronze.employee VALUES
( 1, 'Alice', 25, 'IT'),
( 2, 'Bob', 30, 'HR'),
( 3, 'Charlie', 28, 'Finance'),
( 4, 'David', 32, 'Sales'),
( 5, 'Eve', 27, 'Marketing');

In [0]:
%sql

SELECT * FROM hive_metastore.bronze.employee;

In [0]:
dbutils.fs.ls("dbfs:/user/hive/warehouse/bronze.db/employee")

In [0]:
dbutils.fs.ls("dbfs:/user/hive/warehouse/bronze.db/employee/_delta_log")

In [0]:
dbutils.fs.ls("dbfs:/user/hive/warehouse/bronze.db/employee/_delta_log/00000000000000000000.json")

In [0]:
logs = spark.read.json("dbfs:/user/hive/warehouse/bronze.db/employee/_delta_log/00000000000000000000.json")

In [0]:
logs.show()

In [0]:
%sql
CREATE TABLE hive_metastore.bronze.employee_ext(
  emp_id INT,
  emp_name STRING,
  emp_age INT,
  emp_dept STRING
)
LOCATION 'dbfs:/tmp/employee';

In [0]:
%sql
describe extended hive_metastore.bronze.employee_ext;

In [0]:
%sql
INSERT INTO hive_metastore.bronze.employee_ext VALUES
( 1, 'Alice', 25, 'IT'),
( 2, 'Bob', 30, 'HR'),
( 3, 'Charlie', 28, 'Finance'),
( 4, 'David', 32, 'Sales'),
( 5, 'Eve', 27, 'Marketing');

In [0]:
%sql
SELECT * FROM hive_metastore.bronze.employee_ext;

In [0]:
dbutils.fs.ls("dbfs:/tmp/employee")

In [0]:
%sql
DROP TABLE hive_metastore.bronze.employee;

In [0]:
dbutils.fs.ls("dbfs:/user/hive/warehouse/bronze.db/employee")

In [0]:
%sql
-- For external table
DROP TABLE hive_metastore.bronze.employee_ext;

In [0]:
dbutils.fs.ls("dbfs:/tmp/employee")

In [0]:
%sql
CREATE TABLE hive_metastore.bronze.employee_ext(
  emp_id INT,
  emp_name STRING,
  emp_age INT,
  emp_dept STRING
)
LOCATION 'dbfs:/tmp/employee';

In [0]:
%sql
SELECT * FROM hive_metastore.bronze.employee_ext

I haven't insrted any data in the above table. but the data in te dbfs is not deleted earliar.
So if you want to archive the data then go with external table.

6th session

In [0]:
%sql
DESCRIBE METASTORE;


In [0]:
%sql

CREATE CATALOG dev;

In [0]:
%sql
CREATE EXTERNAL LOCATION `ext_location`
URL 'abfss://container1@sta116.dfs.core.windows.net/data/catalog'
WITH (STORAGE CREDENTIAL ext_catalog_storage);

7th Session

In [0]:
%sql
CREATE CATALOG dev MANAGED LOCATION 'abfss://container1@sta116.dfs.core.windows.net/data/catalog';

In [0]:
%sql
CREATE SCHEMA dev.bronze;

In [0]:
%sql
CREATE TABLE dev.bronze.employee(
  emp_id INT,
  emp_name STRING,
  emp_age INT,
  emp_dept STRING
);

INSERT INTO dev.bronze.employee VALUES 
(1, 'John', 30, 'IT'),
(2, 'Jane', 25, 'HR'),
(3, 'Mike', 35, 'IT'),
(4, 'Sara', 28, 'Finance'),
(5, 'Tom', 32, 'IT');

In [0]:
%sql
DESCRIBE TABLE EXTENDED dev.bronze.employee;